In [ ]:
# Spleeter FAST API

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio, display
import matplotlib.pyplot as plt
import zipfile
import shutil
import tempfile
from pathlib import Path

try:
    import spleeter
    print("✅ Spleeter installation verified!")
except ImportError as e:
    print(f"❌ Spleeter import failed: {e}")

try:
    import librosa
    import soundfile as sf
    print("✅ Audio processing libraries ready!")
except ImportError as e:
    print(f"❌ Audio library import failed: {e}")

print("✅ Setup complete - ready to process audio!")

✅ Spleeter installation verified!
✅ Audio processing libraries ready!
✅ Setup complete - ready to process audio!


In [ ]:
import os
from pathlib import Path

def upload_file_path(file_path):
    """
    Helper function to handle file paths instead of Colab uploads
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    return file_path

print("Upload helper ready ")

Upload helper ready - use upload_file_path('/path/to/file') instead of files.upload()


In [ ]:
from fastapi import FastAPI, File, UploadFile, Form, HTTPException, Request
from fastapi.responses import StreamingResponse, JSONResponse, Response
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import tempfile
import threading
import time
from pathlib import Path
import getpass
import json
import zipfile
import io
import shutil
import librosa
import soundfile as sf
import numpy as np
from spleeter.separator import Separator

nest_asyncio.apply()

auth_token = getpass.getpass("Enter your ngrok authtoken: ")
ngrok.set_auth_token(auth_token)

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize separators 
separators = {
    "2stems-16kHz": Separator('spleeter:2stems-16kHz'),
    "4stems-16kHz": Separator('spleeter:4stems-16kHz'),
    "5stems-16kHz": Separator('spleeter:5stems-16kHz')
}

@app.get("/")
async def root():
    return {"message": "Streaming Spleeter API is running"}

@app.options("/{full_path:path}")
async def options_handler(request: Request):
    return Response(status_code=200, headers={
        "Access-Control-Allow-Origin": "*",
        "Access-Control-Allow-Methods": "GET, POST, PUT, DELETE, OPTIONS",
        "Access-Control-Allow-Headers": "*",
        "Access-Control-Max-Age": "86400",
    })

@app.get("/health")
async def health_check():
    return JSONResponse(content={"status": "healthy"}, headers={
        "Access-Control-Allow-Origin": "*",
        "Access-Control-Allow-Methods": "GET, POST, OPTIONS",
        "Access-Control-Allow-Headers": "*",
    })

def process_audio_with_spleeter(input_file_path, model_name, output_dir):
    """Process audio file with Spleeter Python API"""
    try:
        # Get the separator
        separator = separators[model_name]
        
        # Load the audio file
        waveform, sample_rate = librosa.load(str(input_file_path), sr=44100, mono=False)
        
        # Ensure stereo format (Spleeter expects stereo)
        if waveform.ndim == 1:
            waveform = np.stack([waveform, waveform])
        elif waveform.shape[0] > 2:
            # If more than 2 channels, take first 2
            waveform = waveform[:2]
        
        # Transpose to match Spleeter's expected format (time, channels)
        if waveform.shape[0] == 2:
            waveform = waveform.T
        
        # Separate the audio
        prediction = separator.separate(waveform)
        
        # Create output directory
        stems_dir = output_dir / Path(input_file_path).stem
        stems_dir.mkdir(parents=True, exist_ok=True)
        
        # Save the stems
        for stem_name, stem_data in prediction.items():
            stem_file = stems_dir / f"{stem_name}.wav"
            # stem_data is in format (time, channels), soundfile expects this format
            sf.write(str(stem_file), stem_data, sample_rate)
        
        return stems_dir
        
    except Exception as e:
        raise Exception(f"Spleeter processing failed: {str(e)}")

@app.post("/separate-stream")
async def separate_audio_stream(file: UploadFile = File(...), model_name: str = "2stems-16kHz"):
    valid_models = ["2stems-16kHz", "4stems-16kHz", "5stems-16kHz"]
    if model_name not in valid_models:
        raise HTTPException(status_code=400, detail="Invalid model")

    allowed_extensions = ['.mp3', '.wav', '.flac', '.m4a', '.aac']
    if Path(file.filename).suffix.lower() not in allowed_extensions:
        raise HTTPException(status_code=400, detail="Unsupported file type")

    try:
        temp_dir = Path(tempfile.mkdtemp())
        input_file = temp_dir / file.filename
        with open(input_file, "wb") as buffer:
            buffer.write(await file.read())

        output_dir = temp_dir / "output"
        output_dir.mkdir()
        
        # Use Spleeter Python API
        stems_dir = process_audio_with_spleeter(input_file, model_name, output_dir)
        
        stem_files = list(stems_dir.glob("*.wav"))
        if not stem_files:
            raise HTTPException(status_code=500, detail="No stem files generated")

        stems_data = {f.name: f.read_bytes() for f in stem_files}
        shutil.rmtree(temp_dir)

        def create_multipart_stream(boundary="----WebKitFormBoundary7MA4YWxkTrZu0gW"):
            def generate():
                metadata = {"stems_count": len(stems_data), "stems": [{"name": k, "size": len(v)} for k, v in stems_data.items()]}
                yield f"--{boundary}\r\nContent-Disposition: form-data; name=\"metadata\"\r\nContent-Type: application/json\r\n\r\n{json.dumps(metadata)}\r\n".encode()
                for name, data in stems_data.items():
                    yield f"--{boundary}\r\nContent-Disposition: form-data; name=\"{name}\"; filename=\"{name}\"\r\nContent-Type: audio/wav\r\n\r\n".encode()
                    yield data
                    yield b"\r\n"
                yield f"--{boundary}--\r\n".encode()
            return generate()

        boundary = f"----WebKitFormBoundary{int(time.time())}"
        return StreamingResponse(
            create_multipart_stream(boundary),
            media_type=f"multipart/form-data; boundary={boundary}",
            headers={
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "POST, OPTIONS",
                "Access-Control-Allow-Headers": "*",
                "Content-Disposition": f"attachment; filename=\"{file.filename}_stems.multipart\"",
                "Transfer-Encoding": "chunked"
            }
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Processing error: {str(e)}")

@app.post("/separate-zip")
async def separate_audio_zip(
    request: Request,
    file: UploadFile = File(...),
    model_name: str = Form(default="2stems-16kHz")
):
    # Log request parameters
    request_params = {
        "file_name": file.filename,
        "model_name": model_name,
        "client": request.client.host,
        "user_agent": request.headers.get("user-agent"),
        "headers": dict(request.headers),
    }
    print("ZIP Request Parameters:", json.dumps(request_params, indent=2))

    valid_models = ["2stems-16kHz", "4stems-16kHz", "5stems-16kHz"]
    if model_name not in valid_models:
        raise HTTPException(status_code=400, detail="Invalid model")

    allowed_extensions = ['.mp3', '.wav', '.flac', '.m4a', '.aac']
    if Path(file.filename).suffix.lower() not in allowed_extensions:
        raise HTTPException(status_code=400, detail="Unsupported file type")

    try:
        temp_dir = Path(tempfile.mkdtemp())
        input_file = temp_dir / file.filename
        with open(input_file, "wb") as buffer:
            buffer.write(await file.read())

        output_dir = temp_dir / "output"
        output_dir.mkdir()
        
        stems_dir = process_audio_with_spleeter(input_file, model_name, output_dir)
        
        stem_files = list(stems_dir.glob("*.wav"))
        if not stem_files:
            raise HTTPException(status_code=500, detail="No stem files generated")

        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            for f in stem_files:
                zip_file.write(f, f.name)

        shutil.rmtree(temp_dir)
        zip_buffer.seek(0)

        def generate_zip():
            while True:
                chunk = zip_buffer.read(8192)
                if not chunk:
                    break
                yield chunk

        return StreamingResponse(
            generate_zip(),
            media_type="application/zip",
            headers={
                "Access-Control-Allow-Origin": "*",
                "Content-Disposition": f"attachment; filename=\"{file.filename}_stems.zip\""
            }
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Processing error: {str(e)}")

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=7864, log_level="warning")

ngrok.kill()
threading.Thread(target=run_server, daemon=True).start()
time.sleep(3)

try:
    public_url = ngrok.connect(7864, domain="steady-notable-manatee.ngrok-free.app")
    print(public_url)
except Exception as e:
    print(f"ngrok error: {e}")

2025-07-16 23:14:29.866097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-16 23:14:29.935273: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 23:14:30.307306: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-07-16 23:14:30.307353: W tensorflow/compiler/xla/str

Enter your ngrok authtoken:  ········


2025-07-16 23:14:53.969366: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
t=2025-07-16T23:14:58-0400 lvl=warn msg="ngrok config file found at both XDG and legacy locations, using XDG location" xdg_path=/home/jupyter-nn2415/.config/ngrok/ngrok.yml legacy_path=/home/jupyter-nn2415/.ngrok2/ngrok.yml
t=2025-07-16T23:14:58-0400 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


NgrokTunnel: "https://steady-notable-manatee.ngrok-free.app" -> "http://localhost:7864"
ZIP Request Parameters: {
  "file_name": "Dua Lipa - Levitating Featuring DaBaby (Official Music Video).mp3",
  "model_name": "2stems-16kHz",
  "client": "2600:4808:5392:d500:84d9:6a59:910b:6801",
  "user_agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
  "headers": {
    "host": "steady-notable-manatee.ngrok-free.app",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    "content-length": "5525696",
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "multipart/form-data; boundary=----WebKitFormBoundarynQYBBwAoA7gZtVs5",
    "ngrok-skip-browser-warning": "true",
    "origin": "https://preview.p5js.org",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Not)

t=2025-07-16T23:18:44-0400 lvl=warn msg="Stopping forwarder" name=http-7864-90953414-bfae-459a-b205-7b35d4b09002 acceptErr="failed to accept connection: Listener closed"
t=2025-07-16T23:18:44-0400 lvl=warn msg="Error restarting forwarder" name=http-7864-90953414-bfae-459a-b205-7b35d4b09002 err="failed to start tunnel: session closed"
Process ForkPoolWorker-323:
Process ForkPoolWorker-296:
Process ForkPoolWorker-242:
Process ForkPoolWorker-248:
Process ForkPoolWorker-244:
Process ForkPoolWorker-291:
Process ForkPoolWorker-239:
Process ForkPoolWorker-334:
Process ForkPoolWorker-246:
Process ForkPoolWorker-265:
Process ForkPoolWorker-234:
Process ForkPoolWorker-245:
Process ForkPoolWorker-264:
Process ForkPoolWorker-258:
Process ForkPoolWorker-237:
Process ForkPoolWorker-266:
Process ForkPoolWorker-238:
Process ForkPoolWorker-276:
Process ForkPoolWorker-243:
Process ForkPoolWorker-318:
Process ForkPoolWorker-311:
Process ForkPoolWorker-235:
Process ForkPoolWorker-315:
Process ForkPoolWork